In [1]:
import csv
import collections
import itertools
import numpy as np
import pandas as pd
import string
import re
import os
from sklearn.model_selection import train_test_split  

In [3]:
allevents = os.listdir('Stats/csv')

In [18]:
for eachevent in allevents:
    print("Reading:", eachevent)
    df = pd.read_csv('Stats/csv/'+eachevent)
    df['difference'] = abs(df['Negative(0) class'] - df['Positive(1) class'])
    df['total'] = abs(df['Negative(0) class'] + df['Positive(1) class'])
    print(df.iloc[df['difference'].idxmin()])
    print("------------------------------------------------------------------")
    

Reading: chileEarthquake2014.csv
Class Name           ThirdPartyObservation
Negative(0) class                      157
Positive(1) class                      154
difference                               3
total                                  311
Name: 1, dtype: object
------------------------------------------------------------------
Reading: typhoonYolanda2013.csv
Class Name           Hashtags
Negative(0) class         261
Positive(1) class         303
difference                 42
total                     564
Name: 13, dtype: object
------------------------------------------------------------------
Reading: manilaFloods2013.csv
Class Name           Irrelevant
Negative(0) class           260
Positive(1) class           151
difference                  109
total                       411
Name: 6, dtype: object
------------------------------------------------------------------
Reading: albertaFloods2013.csv
Class Name           ContinuingNews
Negative(0) class               369
Positi

### Unpack GloVe

In [2]:
filename = 'glove.twitter.27B/glove.twitter.27B.100d.txt'

In [3]:
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

In [4]:
twittervocab = loadGloveModel(filename)

Loading Glove Model
Done. 1193515  words loaded!


In [5]:
np.random.seed(120)
randvec = np.random.rand(100)

## Get features

In [6]:
def ptext(text): return re.findall("[a-zA-Z']+", text.lower())

def getwordvec(dataset, maximum_len):
    data = []
    for eachsent in dataset:
        sentence = [twittervocab[word] if word in twittervocab.keys() else randvec for word in eachsent]
        if len(sentence) < maximum_len:
            sentence.extend([[0]*100 for _ in range(maximum_len-len(sentence))])
        data.append(sentence)
    return np.array(data)

def process(filename):
    df = pd.read_csv('Features/'+filename).drop('Unnamed: 0', axis=1)
    common = ['tweetids', 'topics', 'alltopics', 'categories','cleanedtweets']
    columns = [name for name in list(df.columns) if name not in common]
    tweetsdata = df['cleanedtweets'].tolist()
    tweetdata = [ptext(eachtweet) for eachtweet in tweetsdata]
    folder = filename.split('.')[0]
    print("Reading each labels from file: ", filename)
    print("--------------------------------------------------------------")
    os.mkdir('datasets/'+folder)
    
    maximum_len = max([len(i) for i in tweetdata])
    minimum_len = min([len(i) for i in tweetdata])
    print("Maximum length: ", maximum_len)
    print("--------------------------------------------------------------")
    for each_label in columns:
        labels = df[each_label]
        X_train, X_test, y_train, y_test = train_test_split(tweetdata, labels, test_size = 0.20)  
        # train
        x_train = getwordvec(X_train, maximum_len)
        
        new_x_train = np.reshape(x_train, (-1, x_train.shape[1]*x_train.shape[2]))
        # test
        x_test = getwordvec(X_test, maximum_len)

        new_x_test = np.reshape(x_test,(-1, x_test.shape[1]*x_test.shape[2]))
        
        with open('datasets/'+folder+'/'+each_label+'_train_'+str(maximum_len)+'00.txt','w') as fp:
            for left, right in zip(y_train, new_x_train):
                new_right = ' '.join([str(index+1)+':'+str(value) for index, value in enumerate(right)])
                left = -1 if left == 0 else left
                feat = str(left)+' '+str(new_right)
                fp.write("%s\n" %(feat))
                
        with open('datasets/'+folder+'/'+each_label+'_test_'+str(maximum_len)+'00.txt','w') as fp:
            for left, right in zip(y_test, new_x_test):
                new_right = ' '.join([str(index+1)+':'+str(value) for index, value in enumerate(right)])
                left = -1 if left == 0 else left
                feat = str(left)+' '+str(new_right)
                fp.write("%s\n" %(feat))
        print("Finished processing label: ", each_label)
        print("--------------------------------------------------------------")

In [7]:
files = os.listdir('Features/')

In [8]:
for file in files: 
    process(file)
    print("********************************************************************")

Reading each labels from file:  Shooting_features.csv
--------------------------------------------------------------
Maximum length:  29
--------------------------------------------------------------
Finished processing label:  MultimediaShare
--------------------------------------------------------------
Finished processing label:  Hashtags
--------------------------------------------------------------
Finished processing label:  PastNews
--------------------------------------------------------------
Finished processing label:  Sentiment
--------------------------------------------------------------
Finished processing label:  Discussion
--------------------------------------------------------------
Finished processing label:  KnownAlready
--------------------------------------------------------------
Finished processing label:  Factoid
--------------------------------------------------------------
Finished processing label:  EmergingThreats
-------------------------------------------

Finished processing label:  FirstPartyObservation
--------------------------------------------------------------
Finished processing label:  ContinuingNews
--------------------------------------------------------------
Finished processing label:  EmergingThreats
--------------------------------------------------------------
Finished processing label:  Official
--------------------------------------------------------------
Finished processing label:  Hashtags
--------------------------------------------------------------
Finished processing label:  KnownAlready
--------------------------------------------------------------
Finished processing label:  ServiceAvailable
--------------------------------------------------------------
Finished processing label:  Irrelevant
--------------------------------------------------------------
Finished processing label:  MultimediaShare
--------------------------------------------------------------
Finished processing label:  Discussion
--------------

Finished processing label:  KnownAlready
--------------------------------------------------------------
Finished processing label:  EmergingThreats
--------------------------------------------------------------
Finished processing label:  MultimediaShare
--------------------------------------------------------------
Finished processing label:  ServiceAvailable
--------------------------------------------------------------
Finished processing label:  PastNews
--------------------------------------------------------------
Finished processing label:  Sentiment
--------------------------------------------------------------
Finished processing label:  Donations
--------------------------------------------------------------
Finished processing label:  Volunteer
--------------------------------------------------------------
Finished processing label:  Discussion
--------------------------------------------------------------
Finished processing label:  Weather
---------------------------------

Finished processing label:  EmergingThreats
--------------------------------------------------------------
Finished processing label:  InformationWanted
--------------------------------------------------------------
Finished processing label:  ServiceAvailable
--------------------------------------------------------------
Finished processing label:  Donations
--------------------------------------------------------------
Finished processing label:  CleanUp
--------------------------------------------------------------
Finished processing label:  Volunteer
--------------------------------------------------------------
Finished processing label:  GoodsServices
--------------------------------------------------------------
Finished processing label:  PastNews
--------------------------------------------------------------
Finished processing label:  SearchAndRescue
--------------------------------------------------------------
Finished processing label:  Unknown
---------------------------

Finished processing label:  Donations
--------------------------------------------------------------
Finished processing label:  MultimediaShare
--------------------------------------------------------------
Finished processing label:  Volunteer
--------------------------------------------------------------
Finished processing label:  GoodsServices
--------------------------------------------------------------
Finished processing label:  MovePeople
--------------------------------------------------------------
Finished processing label:  Official
--------------------------------------------------------------
Finished processing label:  ServiceAvailable
--------------------------------------------------------------
Finished processing label:  CleanUp
--------------------------------------------------------------
Finished processing label:  InformationWanted
--------------------------------------------------------------
Finished processing label:  SignificantEventChange
-----------------

### Seperating events

In [175]:
featuredata_name = ['Earthquake_features.csv', 'typhoon_features.csv', 'Floods_features.csv']

In [177]:
for f in featuredata_name:
    df = pd.read_csv('Features/'+f).drop('Unnamed: 0', axis=1)
    topics = list(set(df['topics']))
    for i in range(len(topics)):
        print("Reading topic: ", topics[i])
        df[df['topics'] == topics[i]].to_csv('Features/'+topics[i]+'.csv')
    print("-----------------------------------------------------------------------")

Reading topic:  guatemalaEarthquake2012
Reading topic:  chileEarthquake2014
Reading topic:  nepalEarthquake2015
Reading topic:  italyEarthquakes2012
-----------------------------------------------------------------------
Reading topic:  typhoonYolanda2013
Reading topic:  typhoonHagupit2014
-----------------------------------------------------------------------
Reading topic:  queenslandFloods2013
Reading topic:  philipinnesFloods2012
Reading topic:  albertaFloods2013
Reading topic:  manilaFloods2013
-----------------------------------------------------------------------


In [167]:
topics = list(set(df['topics']))